# **College Basketball: An In-depth Study of the "Foul Up 3" Dilemma**

By: Julian Zapata-Hall

---

Below is the code for the analysis portion of my research project. Have allocated a specific folder within this project (Data/CellOTAdjustedWinPercentages/ReproducedCalculations) to ensure reproducibility. This folder will store the results of the calculations you performed while running this notebook. Throughout this notebook, you will have the choice to continue with my original calculation results or the results you have previously generated.

### Installing all Necessary Packages

In [ ]:
import warnings
warnings.filterwarnings('ignore')
%pip install --upgrade pip
%pip install sympy
%pip install pandas 
%pip install numpy
%pip install gdown
%pip install matplotlib

### Coding the 34 Up 3 Fouling Cells

In [ ]:
from sympy import Plane, Point2D, Line2D,Polygon, Segment
#This code creates a polygon for the last 21 seconds of the game clock
#x-axis corresponds with game clock times y-axis to shot clock times
#It also creates 13 lines that cut the polygon in such a way that
#generates the 34 two-dimensional time cells of interest
origpolygon = Polygon([0, 0], [21, 0], [21,21], [0, 0])
line1 = Line2D([3,0],[3,3])
line2 = Line2D([6,0],[6,6])
line3 = Line2D([9,0],[9,9])
line4 = Line2D([12,0],[12,12])
line5 = Line2D([15,0],[15,15])
line6 = Line2D([18,0],[18,18])
line7 = Line2D([3,3],[21,3])
line8 = Line2D([3,0],[33,30])
line9 = Line2D([6,0],[36,30])
line10 = Line2D([9,0],[39 ,30])
line11 = Line2D([12,0],[42 ,30])
line12 = Line2D([15,0],[45 ,30])
line13 = Line2D([18,0],[48 ,30])

up3segments = [origpolygon]
gone_over = [None]
for i in range(3):
  for line in [line1,line2,line3,line4,line5, line6,line7,line8,
               line9,line10,line11,line12,line13]:
    for poli in up3segments:
      if ((poli,line) not in gone_over) and (poli!=None):
        try:
          segment = poli.cut_section(line)
          if None not in segment:
            up3segments.append(segment[0])
            up3segments.append(segment[1])
            up3segments.remove(poli)
          gone_over.append((poli,line))
        except:
          gone_over.append((poli,line))

### Win Percentage Calculating Functions


#### 1. Foul    AND   NOT(Foul)

In [ ]:
import pandas as pd

def regularstrats_winper(in_df,line_list,point_list,polygon,margin,maxreg_secs,
                         minreg_secs,col_x='secs_left_half',col_y1='shot_clock',col_y2='exp_sc',
                         game_id_col='GameID',col_bonustype='BonusType'):
    """A function that calculates 1. the win percentage associated with performing a
    regular non-shooting foul and 2. the win percentage associated with not-fouling
      for a particular cell (given there was a chance to foul). """
    in_df['Polygon'] = polygon
    polygons =  in_df['Polygon'].values
    intersections1 = [str(polygons[i].intersection(line_list[i])) for i in range(len(in_df))]
    in_df['Intersection1'] = intersections1
    intersections2 = [str(polygons[i].intersection(point_list[i])) for i in range(len(in_df))]
    in_df['Intersection2'] = intersections2
    excluded_df = in_df.loc[(in_df['secs_left_reg']<=maxreg_secs)&
                            (in_df['secs_left_reg']>=minreg_secs-30)&
                            ((in_df['team_score'].shift() - in_df['opp_score'].shift())	== margin)&
                            ((in_df['TEAM_FOUL']=='no')|((in_df['TEAM_FOUL']=='yes')&
                                                         (in_df['Intersection2']=='[]')))&
                                                         (in_df['Intersection1']!='[]')].copy()

    df_clean11 = in_df[~in_df['FULLGAMEID'].isin(excluded_df['FULLGAMEID'])].copy()
    indf1 = df_clean11.loc[(df_clean11['TEAM_FOUL']=='yes')&(df_clean11['Intersection2']!='[]')&
                           ((in_df['team_score'].shift() - in_df['opp_score'].shift())	== margin)]
    df_clean12 = df_clean11[df_clean11['FULLGAMEID'].isin(indf1['FULLGAMEID'])].copy()
    df_clean21 = in_df[in_df['FULLGAMEID'].isin(excluded_df['FULLGAMEID'])].copy()
    indf2 = df_clean21.loc[(df_clean21['TEAM_FOUL']=='no')&(df_clean21['Intersection1']!='[]')&
                           ((in_df['team_score'].shift() - in_df['opp_score'].shift())	== margin)]
    df_clean22 = df_clean21[df_clean21['FULLGAMEID'].isin(indf2['FULLGAMEID'])].copy()

    gamesplayed_yes = len(df_clean12['FULLGAMEID'].unique())
    gameswon_yes = (df_clean12.groupby(['FULLGAMEID'])['team_win'].max()).sum()
    win_per_yes = gameswon_yes/gamesplayed_yes
    gamesplayed_no = len(df_clean22['FULLGAMEID'].unique())
    gameswon_no = (df_clean22.groupby(['FULLGAMEID'])['team_win'].max()).sum()
    win_per_no = gameswon_no/gamesplayed_no
    print('Strategic Fouling Area:', polygon)
    print('Win% Yes:', win_per_yes,'Win% No:', win_per_no)
    return polygon,[gameswon_yes,gamesplayed_yes,gameswon_no,gamesplayed_no]

#### 2. Foul + Additional Lane Violation   AND   NOT(Foul + Additional Lane Violation)

In [ ]:
def laneviolationstrats_winper(in_df,line_list,point_list,polygon,margin,maxreg_secs,
                               minreg_secs,col_x='secs_left_half',col_y1='shot_clock',col_y2='exp_sc',
                               game_id_col='GameID',col_bonustype='BonusType'):
    """A function that calculates 1. the win percentage associated with performing a
    regular non-shooting foul and an additional lane violation and
    2. the win percentage associated with not-performing the lane violation approach
     for a particular cell
     (given there was a chance of performing the lane violation approach). """
    in_df['Polygon'] = polygon
    polygons =  in_df['Polygon'].values
    intersections1 = [str(polygons[i].intersection(line_list[i])) for i in range(len(in_df))]
    in_df['Intersection1'] = intersections1
    intersections2 = [str(polygons[i].intersection(point_list[i])) for i in range(len(in_df))]
    in_df['Intersection2'] = intersections2
    excluded_df = in_df.loc[(in_df['secs_left_reg']<=maxreg_secs)&
                            (in_df['secs_left_reg']>=minreg_secs-30)&
                            ((in_df['team_score'].shift() - in_df['opp_score'].shift())	== margin)&
                            ((in_df['BW_FOUL']=='no')|((in_df['BW_FOUL']=='yes')&(in_df['Intersection2']=='[]')))&
                            (in_df['Intersection1']!='[]')].copy()

    ##BW FOUL Is a short way to refer to foul + an additional lane violation
    ##BW are the initials of Bob Walsh CBB Coach credited with the fouling approach
    df_clean11 = in_df[~in_df['FULLGAMEID'].isin(excluded_df['FULLGAMEID'])].copy()
    indf1 = df_clean11.loc[(df_clean11['BW_FOUL']=='yes')&(df_clean11['Intersection2']!='[]')&
                           ((in_df['team_score'].shift() - in_df['opp_score'].shift())	== margin)]
    df_clean12 = df_clean11[df_clean11['FULLGAMEID'].isin(indf1['FULLGAMEID'])].copy()
    df_clean21 = in_df[in_df['FULLGAMEID'].isin(excluded_df['FULLGAMEID'])].copy()
    indf2 = df_clean21.loc[(df_clean21['BW_FOUL']=='no')&(df_clean21['Intersection1']!='[]')&
                           ((in_df['team_score'].shift() - in_df['opp_score'].shift())	== margin)]
    df_clean22 = df_clean21[df_clean21['FULLGAMEID'].isin(indf2['FULLGAMEID'])].copy()

    gamesplayed_yes = len(df_clean12['FULLGAMEID'].unique())
    gameswon_yes = (df_clean12.groupby(['FULLGAMEID'])['team_win'].max()).sum()
    win_per_yes = gameswon_yes/gamesplayed_yes
    gamesplayed_no = len(df_clean22['FULLGAMEID'].unique())
    gameswon_no = (df_clean22.groupby(['FULLGAMEID'])['team_win'].max()).sum()
    win_per_no = gameswon_no/gamesplayed_no
    print('Strategic Fouling Area:', polygon)
    print('Win% Yes:', win_per_yes,'Win% No:', win_per_no)
    return polygon,[gameswon_yes,gamesplayed_yes,gameswon_no,gamesplayed_no]

In [ ]:
import numpy as np
import gdown
import warnings
warnings.filterwarnings('ignore')

##These urls correspond to the play by play data from the
##2009-10 and 2010-11 seasons, 2011-12 and 2012-13 seasons, 2013-14 seasons and 2014-15 seasons,
##2015-16 season, 2016-17 season, 2017-18 and 2018-19 seasons, 2019-20 and
##2020-21 and 2021-22 seasons of NCAA Division 1 Mens Basketball respectively.

urls = ['https://drive.google.com/file/d/1PxcGXvscaoaFPksHAabhLoIlxopsBUr3/view?usp=sharing',
        'https://drive.google.com/file/d/1qOgZ28TZTNUY4BjoeRZCnTGLUyTdDnIo/view?usp=sharing',
        'https://drive.google.com/file/d/1eUbCluRtm1NOZjSbQYR1EjkNs_hlR8pJ/view?usp=sharing',
        'https://drive.google.com/file/d/1_EELPYHrpTJVLJY_QkAESKa8w8EkPmzE/view?usp=sharing',
        'https://drive.google.com/file/d/1SC3yIfTWMp5Fe6lh-zzdssXFE3gFWY9e/view?usp=sharing',
        'https://drive.google.com/file/d/1k2mn1LQd2J1gA5f4vXlG62sxdDptD8HB/view?usp=sharing',
        'https://drive.google.com/file/d/152fSRK4ny7HD7so331YR2JzhPsjdXFEI/view?usp=sharing']



years = [2010,2012,2014,2016,2017,2018,2020] ##ordered years urls correspond to

##This analysis is time consuming and can take up to 9 hours to compute
##It was originally run in 14 different notebook files simultaneously, saving time.
##However, these files have been integrated to better display the analysis roadmap.
for u in range(len(urls)):
  url = urls[u]
  url='https://drive.google.com/uc?id=' + url.split('/')[-2]
  pseudo_df = gdown.download(url, "current_pbp_file", quiet=False)
  pbp_data = pd.read_csv(pseudo_df)
  ##This  code corrects for overtime.
  ##If a game goes to OT 0.5 wins are assigned to each team.
  ##This reduces the effect of random noise on our calculated win percentages.
  pbp_data['half'] = (pbp_data['half'].astype(int))
  pbp_data['MAX_HALF'] = pbp_data.groupby(['FULLGAMEID'])['half'].transform(max)
  pbp_data['OT'] = pbp_data['MAX_HALF'] > 2
  pbp_data['team_win'] = [pbp_data['team_win'][i] if pbp_data['OT'][i] == False else .5 for i in range(len(pbp_data['OT']))]
  ##This code checks if there was an opportunity to commit a regular foul
  ##or an opportunity to commit a foul and a lane violation and
  ##creates a tag for if such a foul was committed.
  pbp_data['FTA'] = pbp_data['play_type'] =='free throw'
  pbp_data['Time'] = pbp_data['FULLGAMEID'] + pbp_data['FULLGAMEID'] + \
    pbp_data['half'].astype(str) + 'h' + pbp_data['secs_left_half'].astype(str)
  pbp_data['No. FTA'] = pbp_data.groupby('Time')['FTA'].transform('sum')
  pbp_data['FTA No.'] = pbp_data.groupby('Time')['FTA'].transform('cumsum')
  pbp_data['FTA1made'] = (pbp_data['FTA No.'] == 1)&(pbp_data['scoring_play']==1)
  pbp_data['FTA2made'] = (pbp_data['FTA No.'] == 2)&(pbp_data['scoring_play']==1)
  pbp_data['firstftmade'] = pbp_data.groupby('Time')['FTA1made'].transform('sum')
  pbp_data['secondftmade'] = pbp_data.groupby('Time')['FTA2made'].transform('sum')
  filters = [
        ((pbp_data['TEAM_FOUL']=='yes') &
          ((pbp_data['team_score'].shift() - pbp_data['opp_score'].shift())	== 3))&
                                            ((pbp_data['team_score'].shift() - pbp_data['opp_score'].shift())	== 3) &
                                            ((pbp_data['No. FTA']== 1)|(pbp_data['No. FTA']== 2)) & 
                                            ((pbp_data['firstftmade'] == 0)|(pbp_data['secondftmade'] == 1)),
      ((pbp_data['TEAM_FOUL']=='no') &
       ((pbp_data['team_score'].shift() - pbp_data['opp_score'].shift())	== 3))|
       ((pbp_data['TEAM_FOUL']=='yes') & ((pbp_data['team_score'].shift() - pbp_data['opp_score'].shift())	== 3) & 
        (pbp_data['No. FTA']== 2)& (pbp_data['firstftmade'] == 1)& (pbp_data['secondftmade']== 0))
  ]
  values = ["yes", "no"]
  ##BW FOUL Is a short way to refer to foul + an additional lane violation
  ##BW are the initials of Bob Walsh CBB Coach credited with the fouling approach
  pbp_data["BW_FOUL"] = np.select(filters, values, default="na")


  ##This code chunk stores every single play in the season (in the for loop)
  ##as a translation between two coordinates(game_clock_start,shot_clock_start)
  ## and (game_clock_end,shot_clock_end) as well as the end point.
  ##Namely, (game_clock_end,shot_clock_end) where start means start of the event,
  ##which corresponds which the aftermath of the last play and end which
  ##means the end of the event or
  ##the instant before the current event was recorded.

  ptmargin = 3
  polygons = up3segments
  max_gc = int(max([polygon.bounds[2] for polygon in polygons]))
  lv_included_df =  pbp_data.loc[(pbp_data['secs_left_reg']<=max_gc)&
                                 ((pbp_data['team_score'].shift() - pbp_data['opp_score'].shift())== ptmargin)&
                                 (pbp_data['half']==2)&((pbp_data['BW_FOUL']=='yes')|(pbp_data['BW_FOUL']=='no'))].copy()
  lv_df = pbp_data[(pbp_data['FULLGAMEID'].isin(lv_included_df['FULLGAMEID']))&
                   (pbp_data['secs_left_reg']<=max_gc)].copy()
  reg_included_df =  pbp_data.loc[(pbp_data['secs_left_reg']<=max_gc)&
                                  ((pbp_data['team_score'].shift() - pbp_data['opp_score'].shift())== ptmargin)&
                                  (pbp_data['half']==2)&
                                  ((pbp_data['TEAM_FOUL']=='yes')|(pbp_data['TEAM_FOUL']=='no'))].copy()
  reg_df = pbp_data[(pbp_data['FULLGAMEID'].isin(reg_included_df['FULLGAMEID']))&(pbp_data['secs_left_reg']<=max_gc)].copy()
  reg_lines = []##stores the traslations in the time plane for each play as a line segment
  reg_points = []##stores the game clock and shot clock times just before each event occurs
  for i in range(len(reg_df)):
    a = reg_df['secs_left_half'].shift().fillna(12000).values[i]
    b = reg_df['shot_clock'].shift().fillna(30).values[i]
    c = reg_df['secs_left_half'].values[i]
    d = reg_df['exp_sc'].values[i]
    if (a==c) and (b==d):
      reg_lines.append(Point2D(a,b))
    else:
      reg_lines.append(Segment(Point2D(a,b),Point2D(c,d)))
    reg_points.append(Point2D(c,d))
  reg_df['Lines']=reg_lines
  reg_df['Points']=reg_points
  lv_lines = []##stores the traslations in the time plane for each play as a line segment
  lv_points = []##stores the game clock and shot clock times just before each event occur
  for i in range(len(lv_df)):
    a = lv_df['secs_left_half'].shift().fillna(12000).values[i]
    b = lv_df['shot_clock'].shift().fillna(30).values[i]
    c = lv_df['secs_left_half'].values[i]
    d = lv_df['exp_sc'].values[i]
    if (a==c) and (b==d):
      lv_lines.append(Point2D(a,b))
    else:
      lv_lines.append(Segment(Point2D(a,b),Point2D(c,d)))
    lv_points.append(Point2D(c,d))
  lv_df['Lines']=lv_lines
  lv_df['Points']=lv_points


  ##This code chunk calculates games played and games won
  ##for each fouling approach associated
  ##with each of the given time cells during this season (in the for loop)
  strats = [] ##stores the cells in polygon format
  gws_foul_reg = []
  gps_foul_reg = []
  gws_nofoul_reg= []
  gps_nofoul_reg = []
  gws_foul_laneviolation = []
  gps_foul_laneviolation = []
  gws_nofoul_laneviolation = []
  gps_nofoul_laneviolation = []
  print('Up by '+str(ptmargin)+' scenario!')
  print('Start:')
  print()
  for i in range(len(polygons)):
    polygon = polygons[i]
    nmax_gc = polygon.bounds[2]
    nmin_gc = polygon.bounds[0]
    strat, [f_gw_reg,f_gp_reg,nf_gw_reg,nf_gp_reg] = regularstrats_winper(reg_df,reg_lines,
                                                                          reg_points,polygon,ptmargin,
                                                                          nmax_gc,nmin_gc)
    strat, [f_gw_lv,f_gp_lv,nf_gw_lv,nf_gp_lv] = laneviolationstrats_winper(lv_df,lv_lines,lv_points,
                                                                            polygon,ptmargin,
                                                                            nmax_gc,nmin_gc)
    strats.append(polygon)
    gws_foul_reg.append(f_gw_reg)
    gps_foul_reg.append(f_gp_reg)
    gws_nofoul_reg.append(nf_gw_reg)
    gps_nofoul_reg.append(nf_gp_reg)
    gws_foul_laneviolation.append(f_gw_lv)
    gps_foul_laneviolation.append(f_gp_lv)
    gws_nofoul_laneviolation.append(nf_gw_lv)
    gps_nofoul_laneviolation.append(nf_gp_lv)
    print()

  ##This code chunk creates and saves the within cell calculations for the
  ##regular and lane violation approaches in a reproduced calculations folder
  ##that way, there exist Original and Reproduced Calculations that can be
  ##compared to ensure verification and reproducibility.
  regular_resultdf = pd.DataFrame({'Strategy':strats,'Games Won Foul':gws_foul_reg,
                                   'Games Played Foul':gps_foul_reg,
                                   'Games Won NoFoul':gws_nofoul_reg,
                                   'Games Played NoFoul':gps_nofoul_reg})
  laneviolation_resultdf = pd.DataFrame({'Strategy':strats,
                                         'Games Won Foul':gws_foul_laneviolation,
                                         'Games Played Foul':gps_foul_laneviolation,
                                         'Games Won NoFoul':gws_nofoul_laneviolation,
                                         'Games Played NoFoul':gps_nofoul_laneviolation})
  regular_resultdf.to_csv('./Data/CellOTAdjustedWinPercentages/ReproducedCalculations/rep_reg_pbp_'+
                          str(years[u])+'_results.csv',index=False)
  laneviolation_resultdf.to_csv('./Data/CellOTAdjustedWinPercentages/ReproducedCalculations/rep_lv_pbp_'+
                                str(years[u])+'_results.csv',index=False)


## Resulting Fouling Strategy Description

This is my recommended strategy. The numbers within the cells are merely cell identifying numbers ranging from 14 to 34; see Figure 1. Firstly, it must be noted that the final 12 seconds remaining on the game clock match previous conceptions of when a foul should be committed.
Interestingly, however, there are more green cells as the time left on the game clock increases. This suggests that fouling earlier may be beneficial. Additionally, when considering this graph alongside the two fouling decision graphs, a pattern emerges. Figure 7 shows that the overtime-adjusted win percentages associated with fouling remain constant between 87% to 93%. Figure 8 shows that the overtime-adjusted win percentages associated with not fouling dip under 80% as the shot clock decreases. Thus, fouling in the cells with a low shot clock (e.g., cell 22 and cell 27), results in the highest competitive benefit. 


## Concluding Statements


Through my analysis, I have been able to generate a fouling strategy for up by 3 that any college can implement. Moreover, by using a new variable that had not previously been employed and an extensive amount of game data, I have been able to shine light on the most debated question within strategic fouling. Although this set of results should not be applied outside of the scope of NCAA college basketball, a similar project could elevate the level of fouling strategy within the NBA or any other basketball league around the world.



## Acknowledgments

Thank you to these great people and their constant feedback, validation and overall support:

-	Bob Walsh (college coach, St John’s University Red Storm)
-	Chris Murphy (college coach, Union College Garnet Chargers)
-	Dr. Laurie Shaw (data scientist, Manchester City)
-	Dr. Roger Hoerl (statistician, Union College)
-	Dr. Sebastian Kurtek (statistician, The Ohio State University)
-	Dr. Wayne Winston (statistician, Indiana University)
-	Ken Pomeroy (statistician, kenpom.com)
-	Kevin Weckworth (college coach, Hartford Hawks)
-	Sudarshan Gopaladesikan (data scientist, Atalanta BC)
-	163 NCAA Men’s Basketball Coaches / Anonymous Survey Responses.
